In [117]:
import fastText
import math
import numpy as np 
from numpy import random
from random import sample
from keras.models import Sequential, Model
from keras.callbacks import ModelCheckpoint
from keras.layers import *
from keras import *
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
import re
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score, mean_squared_error

In [2]:
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True
set_session(tf.Session(config=config))

In [3]:
ft = fastText.load_model("/home1/zishan/raghav/wiki.hi.bin")

nb_embedding_dims = ft.get_dimension()
nb_sequence_length = 75

In [4]:
def twitter_tokenizer(textline):
    textLine = re.sub(r'http\S+', 'URL', textline)
    textline = re.sub('@[\w_]+', 'USER_MENTION', textline)
    textline = re.sub('\|LBR\|', '', textline)
    textline = re.sub('\.\.\.+', '...', textline)
    textline = re.sub('!!+', '!!', textline)
    textline = re.sub('\?\?+', '??', textline)
    words = re.compile('[\U00010000-\U0010ffff]|[\w-]+|[^ \w\U00010000-\U0010ffff]+', re.UNICODE).findall(textline.strip())
    words = [w.strip() for w in words if w.strip() != '']
    return(words)

In [5]:
word_vectors_ft = {}
def process_features(textline, nb_sequence_length, nb_embedding_dims, tokenize=True):
    if not tokenize:
        words = textline.split()
    else:
        words = twitter_tokenizer(textline)
    features_ft = np.zeros((nb_sequence_length, nb_embedding_dims))
    features_idx = np.zeros(nb_sequence_length)
    max_words = min(len(words), nb_sequence_length)
    idx = nb_sequence_length - len(words[:max_words])
    for w in words[:max_words]:
        if w in word_vectors_ft:
            wv = word_vectors_ft[w]
        else:
            wv = ft.get_word_vector(w.lower())
            word_vectors_ft[w] = wv
        features_ft[idx] = wv
        
        idx = idx + 1
    return features_ft

In [7]:
def sequential_generator(filename, 
                         batch_size, 
                         labels2Idx:'dict to make output labels',
                         check:'to check if all lines in file are of same length.To check enter the len of line after splitting it by tabs' = None,
                         tokenize:'specify if using twitter tokenzor to preprocess lines'=False, 
                        ):    
    
    f = open(filename)
    n_labels = len(labels2Idx)
    while True:
        batch_features_ft = np.zeros((batch_size, nb_sequence_length, nb_embedding_dims))
        batch_labels = np.zeros((batch_size, len(labels2Idx)))
        batch_intensity = np.zeros((batch_size, 1))
        for i in range(batch_size):
            line = f.readline()
            if ("" == line):
                f.seek(0)
                line = f.readline()
            data = line.strip().split('\t')
            if check:
                if len(data)!=check:
                    i-=1
                    continue
            batch_features_ft[i] = process_features(data[0], nb_sequence_length, nb_embedding_dims, tokenize= tokenize)
            batch_labels[i] = to_categorical(labels2Idx[data[1]], n_labels)
            batch_intensity[i] = float(data[2])
        yield ([batch_features_ft], [batch_labels, batch_intensity])

In [ ]:
train_file = '/home1/zishan/raghav/Data/train_with_intensity.txt'
weights_file ='/home1/zishan/raghav/weights/bilstm_3cnn_with_intensity.h5'
log_file = '/home1/zishan/raghav/logs/bilstm_3cnn_with_intensity.txt'
batch_size = 16
check_for_generator = 2
labels2Idx = labels2Idx
tokenize = True
samples_per_epoch = len(train_sentences)
steps_per_epoch = math.ceil(samples_per_epoch / batch_size)
for i in sequential_generator(filename = train_file, batch_size = batch_size, check = check_for_generator, 
                                             labels2Idx= labels2Idx, tokenize= tokenize):
    x = 1

In [36]:
def train_dev_sentences(filetrain, filedev, check:'to check if lines of file are all same lenght after separating by tab'):
    labels2Idx = {}
    train_lines = [line.strip().split("\t") for line in open(filetrain) if len(line.split('\t'))==check]
    dev_lines = [line.strip().split("\t") for line in open(filedev) if len(line.strip().split('\t'))==check]
    for dataset in [train_lines, dev_lines]:
        for line in dataset:
            label = line[1]
            if label not in labels2Idx.keys():
                labels2Idx[label]= len(labels2Idx)
    train_sentences =[]
    train_labels = []
    dev_sentences = []
    dev_labels = []
    try:
        for line_no, x in enumerate(train_lines):
#             print(x, line_no)
            train_sentences.append(x[0])
            train_labels.append([labels2Idx[x[1]], float(x[2])])
    except Exception as e:
        print(x, line_no, end='\n\n')
#         continue
    try:
        for line_no, x in enumerate(dev_lines):
            dev_sentences.append(x[0])
            dev_labels.append([labels2Idx[x[1]], float(x[2])])
    except Exception as e:
        print(x, line_no)
    return (train_sentences, train_labels, dev_sentences, dev_labels, labels2Idx)

In [37]:
train_file = '/home1/zishan/raghav/Data/train_with_intensity.txt'
dev_file = '/home1/zishan/raghav/Data/dev_with_intensity.txt'
train_sentences, train_labels, dev_sentences, dev_labels, labels2Idx = train_dev_sentences(train_file, dev_file, 3)

In [38]:
n_words = 0
for sentence in train_sentences:
    n_words+=len(sentence)
print(n_words)

111978


In [39]:
from collections import Counter
print(Counter([x[0] for x in train_labels]))
print(labels2Idx)

Counter({0: 389, 2: 191, 4: 146, 5: 146, 1: 119, 3: 85, 6: 61, 7: 47, 8: 19})
{'SADNESS': 0, 'FEAR/ANXIETY': 1, 'SYMPATHY/PENSIVENESS': 2, 'JOY': 3, 'OPTIMISM': 4, 'NO-EMOTION': 5, 'DISGUST': 6, 'ANGER': 7, 'SURPRISE': 8}


In [41]:
print(train_sentences[:2])
print(train_labels[:10])
print(labels2Idx)
print(len(train_labels))

['आतंंकवादियों ने सुबह करीब सवा छह बजे पीछे से 3-1 गोरखा राइफल्स के शिविर को निशाना बनाते हुए अंधाधुंध गोलीबारी की जिसमें एक आम नागरिक की मौत हो गई जबकि एक जवान घायल हो गया।', 'दक्षिण अफ्रीका के डरबन शहर में गुरुवार को कुछ अज्ञात हमलावरों ने एक मस्जिद पर हमला कर मस्जिद के इमाम की हत्या कर दी और 2 अन्य लोगों को चाकू घोंपकर घायल कर दिया।']
[[0, 0.4], [0, 0.3], [1, 0.4], [2, 0.6], [0, 0.8], [3, 0.3], [0, 1.0], [4, 0.3], [4, 0.5], [1, 0.4]]
{'SADNESS': 0, 'FEAR/ANXIETY': 1, 'SYMPATHY/PENSIVENESS': 2, 'JOY': 3, 'OPTIMISM': 4, 'NO-EMOTION': 5, 'DISGUST': 6, 'ANGER': 7, 'SURPRISE': 8}
1203


In [42]:
n_labels = len(labels2Idx)

In [91]:
def compile_model_bilstm_cnn(no_labels:'total labels for classification'):
    model_input_embedding = Input(shape = (nb_sequence_length, nb_embedding_dims))
    lstm_block = Bidirectional(LSTM(100, dropout = 0.5, return_sequences=True))(model_input_embedding)
    lstm_block = LeakyReLU()(lstm_block)

    filter_sizes = (3, 4, 5)
    conv_blocks = []
    for sz in filter_sizes:
        conv = Conv1D(
            filters = 200,
            kernel_size = sz,
            padding = 'valid',
            strides = 1
        )(lstm_block)
        conv = LeakyReLU()(conv)
        conv = GlobalMaxPooling1D()(conv)
        conv = Dropout(0.5)(conv)
        conv_blocks.append(conv)
    model_concatenated = concatenate([conv_blocks[0], conv_blocks[1], conv_blocks[2]])
    model_concatenated = Dense(100)(model_concatenated)
    model_concatenated = LeakyReLU()(model_concatenated)
    model_output_classification = Dense(no_labels, activation = "softmax", name='for_classification')(model_concatenated)
    model_output_intensity = Dense(1, activation = "sigmoid", name='for_intensity')(model_concatenated)
    new_model = Model(model_input_embedding, [model_output_classification, model_output_intensity])
    new_model.compile(loss=['categorical_crossentropy', 'mse'], optimizer='nadam', metrics = ['accuracy'])
    new_model.summary()
    return new_model

In [118]:
model = compile_model_bilstm_cnn(no_labels = n_labels)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 75, 300)      0                                            
__________________________________________________________________________________________________
bidirectional_4 (Bidirectional) (None, 75, 200)      320800      input_4[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_16 (LeakyReLU)      (None, 75, 200)      0           bidirectional_4[0][0]            
__________________________________________________________________________________________________
conv1d_10 (Conv1D)              (None, 73, 200)      120200      leaky_re_lu_16[0][0]             
__________________________________________________________________________________________________
conv1d_11 

In [89]:
train_file = '/home1/zishan/raghav/Data/train_with_intensity.txt'
weights_file ='/home1/zishan/raghav/weights/bilstm_3cnn_with_intensity.h5'
log_file = '/home1/zishan/raghav/logs/bilstm_3cnn_with_intensity.txt'
batch_size = 16
check_for_generator = 2
labels2Idx = labels2Idx
tokenize = True
samples_per_epoch = len(train_sentences)
steps_per_epoch = math.ceil(samples_per_epoch / batch_size)

In [119]:
generator = sequential_generator(filename=train_file, batch_size=len(train_sentences), labels2Idx=labels2Idx)


In [120]:
x,y = generator.__next__()

In [121]:
print(x[0].shape)
print(y[0].shape)
print(y[1].shape)

(1203, 75, 300)
(1203, 9)
(1203, 1)


In [122]:
for i in range(20):
    model.fit(x, y, epochs=1)
    
    testset_features = np.zeros((len(dev_sentences), nb_sequence_length, nb_embedding_dims))
    for i in range(len(dev_sentences)):
        testset_features[i] = process_features(dev_sentences[i], nb_sequence_length, nb_embedding_dims)
    results = model.predict(testset_features)
    class_pred = results[0].argmax(axis=1)
    intensity_pred = results[1]
    true_classes= [label[0] for label in dev_labels]
    true_intensity = [label[1] for label in dev_labels]
    mse = mean_squared_error(true_intensity, intensity_pred)
    f1 = f1_score(true_classes, class_pred, average='macro')
    print(f1, mse)
    

Epoch 1/1
1203/1203 [==============================] - 16s 13ms/step - loss: 1.9597 - for_classification_loss: 1.8896 - for_intensity_loss: 0.0700 - for_classification_acc: 0.3242 - for_intensity_acc: 0.0865


/home1/zishan/anaconda3/envs/raghav_btp/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.060589643546476474 0.0653578128398265
Epoch 1/1
1203/1203 [==============================] - 12s 10ms/step - loss: 1.7877 - for_classification_loss: 1.7234 - for_intensity_loss: 0.0642 - for_classification_acc: 0.3965 - for_intensity_acc: 0.1097
0.05625174046226679 0.06641256494982978
Epoch 1/1
1203/1203 [==============================] - 12s 10ms/step - loss: 1.6843 - for_classification_loss: 1.6223 - for_intensity_loss: 0.0620 - for_classification_acc: 0.3940 - for_intensity_acc: 0.1180
0.09970531969006416 0.06881109819545463
Epoch 1/1
1203/1203 [==============================] - 12s 10ms/step - loss: 1.6113 - for_classification_loss: 1.5504 - for_intensity_loss: 0.0609 - for_classification_acc: 0.4198 - for_intensity_acc: 0.1214
0.05592386474562209 0.0666635274380257
Epoch 1/1
1203/1203 [==============================] - 12s 10ms/step - loss: 1.5137 - for_classification_loss: 1.4553 - for_intensity_loss: 0.0583 - for_classification_acc: 0.4688 - for_intensity_acc: 0.1272
0.0683556

In [ ]:
max_f1 = 0
for epoch in range(200):
    print("Epoch {}".format(epoch))
    model.fit_generator(sequential_generator(filename = train_file, batch_size = batch_size, check = check_for_generator, 
                                             labels2Idx= labels2Idx,tokenize= tokenize),
                        steps_per_epoch= steps_per_epoch, epochs=1,)

    testset_features = np.zeros((len(dev_sentences), nb_sequence_length, nb_embedding_dims))
    for i in range(len(dev_sentences)):
        testset_features[i] = process_features(dev_sentences[i], nb_sequence_length, nb_embedding_dims)
    results = model.predict(testset_features)


    predLabels = results.argmax(axis=-1)
    devLabels = dev_labels
    f1 = f1_score(devLabels, predLabels, average='macro') # offensive is the major class. So other is minor
    r = recall_score(devLabels, predLabels, average='macro')
    p = precision_score(devLabels, predLabels, average='macro')
    a = accuracy_score(devLabels, predLabels)
    
    if f1> max_f1:
        model.save_weights(weights_file)
        with open(log_file,'a+') as f:
            text = str(epoch)+', a: '+str(a) +', f1:' +str(f1) +'\n'
            f.write(text)
        max_f1 = f1

    print(a,f1)

# transfer learning

In [ ]:
def test_model_tl_unfreezing(generator, 
               train_sentences, 
               devLabels, 
               number_of_tests,
               number_of_epochs,
               filename_to_log, 
               labels2Idx,
               filename_to_save_weigths,
               batch_size, 
               unfreezing_strategy: 'list containing a tuple of indices to unfreeze at each step',
               train_file:'filepath for traininig',
               f1_measure:'binary/macro etc', 
               pos_label:'only if binary f1',
               load_model_weights=False,
               model_weights_file:'give filepath as str'=None, 
               tokenize=True,
               nb_sequence_length = nb_sequence_length,
               nb_embedding_dims= nb_embedding_dims, 
               check_for_generator=None):
    
    f = open(filename_to_log, 'w', encoding='utf-8')
    f.close()
   
    total_f1=0
    total_prec=0
    total_acc=0
    total_recall=0
    
    for test_number in range(number_of_tests):
        print("Test %d/%d" %(test_number+1, number_of_tests))
        model = compile_model_bilstm_cnn(4)

        # transfer learning
        if load_model_weights and model_weights_file:
                model.load_weights(model_weights_file)
                print("removing top layer")
                model.layers.pop()
                output = Dense(len(labels2Idx), activation = 'softmax')(model.layers[-1].output)
                final_model = Model(inputs=model.input, outputs=[output])

        samples_per_epoch = len(train_sentences)
        epochs = number_of_epochs
        batch_size = batch_size
        steps_per_epoch = math.ceil(samples_per_epoch / batch_size)

        max_f1=0
        max_p=0
        max_r=0
        max_a=0
        
        # load pretrained weights
        # model.compile
        # save tmp weights
        # iterate over layers
        #    load tmp weights
        #    iterate over epochs
        #        unfreeze top frozen layer
        #        save best model as tmp weights
        
        
        final_model.save(filename_to_save_weigths)
        
        # layers_to_unfreeze = [18, 16, 3, 1]
        
        for ulayer in unfreezing_strategy:
            print("unfreezing " + final_model.layers[ulayer[0]].name)
            print("---------------------------------------")
            final_model.load_weights(filename_to_save_weigths)            
            for i, layer in enumerate(final_model.layers):
                
                # TF strategy: gradual unfreezing
                #if i >= ulayer:
                #    layer.trainable = True
                #else:
                #    layer.trainable = False
                # 
                ## TF strategy: single
                
                if i >= ulayer[1] and i <= ulayer[0]:
                    layer.trainable = True
                else:
                    layer.trainable = False
                    
                print(str(i) + ' ' + layer.name + ' ' + str(layer.trainable))
            final_model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
        
            for epoch in range(epochs):
                print("Epoch: %d/%d" %(epoch+1, epochs))
                final_model.fit_generator(
                    generator(filename = train_file, batch_size = batch_size, check = check_for_generator, 
                              labels2Idx= labels2Idx,tokenize= tokenize), 
                    steps_per_epoch= steps_per_epoch, epochs=1
                )

                testset_features = np.zeros((len(dev_sentences), nb_sequence_length, nb_embedding_dims))
                for i in range(len(dev_sentences)):
                    testset_features[i] = process_features(dev_sentences[i], nb_sequence_length, nb_embedding_dims)
                results = final_model.predict(testset_features)

                predLabels = results.argmax(axis=-1)
                devLabels = devLabels
                f1 = f1_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label) # offensive is the major class. So other is minor
                r = recall_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
                p = precision_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
                a = accuracy_score(devLabels, predLabels)
                if max_f1 < f1:
                    print("model saved. F1 is %f" %(f1))
                    final_model.save(filename_to_save_weigths)
                    max_f1 = f1
                    max_p = p
                    max_r = r
                    max_a = a
                text = "prec: "+ str(p)+" rec: "+str(r) +" f1: "+str(f1) +" acc: "+str(a)+" \n"
                print("Test-Data: Prec: %.3f, Rec: %.3f, F1: %.3f, Acc: %.3f" % (p, r, f1, a))
        to_write= "prec: "+ str(max_p)+" rec: "+str(max_r) +" f1: "+str(max_f1) +" acc: "+str(max_a)+" \n"
        print(to_write)
        with open(filename_to_log,'a') as f:
            f.write(to_write)
        total_f1+=max_f1
        total_prec+=max_p
        total_acc+=max_a
        total_recall+=max_r    
        print("*****************************************************************************")
    final_text = "avg_prec: " +str(total_prec/number_of_tests)+" total_rec: "+str(total_recall/number_of_tests) +" total_f1: "+str(total_f1/number_of_tests) +" total_acc: "+str(total_acc/number_of_tests)+" \n"
    print(final_text)
    with open(filename_to_log,'a') as f:
        f.write(final_text)

In [ ]:
# list of tuples. Every tuple contains range of layers which need to be unfrozen. Rest all are frozen
single_unfreeze_bottom_up = [(18, 18), (17, 16), (15, 3), (2, 1), (18,1)] 
single_unfreeze_top_down = [(18, 18),   (2, 1),(15, 3), (17, 16), (18,1)]
all_unfreeze = [(18,1)]
gradual_unfreezing = [(18,18), (18,16), (18,3), (18,1)]

strings =['suf_bu', 'suf_td','all_unfreeze','gradual_unfreeze']
# strings=['gradual_unfreeze']
unfreeze_strategy = [single_unfreeze_bottom_up, single_unfreeze_top_down, all_unfreeze, gradual_unfreezing]
# unfreeze_strategy = [gradual_unfreezing]

In [ ]:
for i in range(len(strings)):
    string = strings[i]
    print("approach: %s" %(string))
    
    generator = sequential_generator
    train_sentences = train_sentences
    devLabels = dev_labels
    number_of_tests = 1
    number_of_epochs = 200
    labels2Id = labels2Idx
    log_file = '/home1/zishan/raghav/logs/tl_bilstm_3cnn_dropout=0.8_' +string+'.txt' 
    print("log file: %s" %(log_file))
    weights_file='/home1/zishan/raghav/weights/tl_bilstm_3cnn_dropout=0.8_'+string+'.h5'
    print("save weights file: %s" %(weights_file))
    batch_size=16
    train_file='/home1/zishan/raghav/Data/train_31_aug.txt'
    f1_measure='macro'
    pos_label=1
    strategy = unfreeze_strategy[i]
    print(strategy)
    load_model_weights=True
    load_weights_file = '/home1/zishan/raghav/weights/pretrain_bilstm_3cnn_dropout=0.5.h5'
    nb_sequence_length = nb_sequence_length
    nb_embedding_dims= nb_embedding_dims
    check_for_generator=2
    
    test_model_tl_unfreezing(generator=generator, 
           train_sentences=train_sentences, 
           devLabels=devLabels, 
           number_of_tests= number_of_tests,
           number_of_epochs=number_of_epochs, 
           filename_to_log=log_file, 
           labels2Idx = labels2Id,
           filename_to_save_weigths=weights_file,
           batch_size=batch_size,
           unfreezing_strategy = strategy,       
           train_file=train_file, 
           f1_measure=f1_measure, 
           pos_label=pos_label, 
           load_model_weights=True,
           model_weights_file = load_weights_file, 
           nb_sequence_length=nb_sequence_length, 
           nb_embedding_dims=nb_embedding_dims, 
           check_for_generator= check_for_generator)

In [ ]:
model = compile_model_bilstm_cnn(4)

In [ ]:
model.load_weights('/home1/zishan/raghav/weights/pretrain_bilstm_3cnn_dropout=0.8.h5')

In [ ]:
model.layers.pop()

In [ ]:
output = Dense(n_labels, activation='softmax')(model.layers[-1].output)

In [ ]:
final_model = Model(inputs=model.input, outputs=[output])

In [ ]:
final_model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])

In [ ]:
train_file = '/home1/zishan/raghav/Data/train_31_aug.txt'
weights_file ='/home1/zishan/raghav/weights/tl_bilstm_3cnn_dropout=0.8.h5'
log_file = '/home1/zishan/raghav/logs/tl_bilstm_3cnn_dropout=0.8.txt'
batch_size = 16
check_for_generator = 2
labels2Idx = labels2Idx
tokenize = True
samples_per_epoch = len(train_sentences)
steps_per_epoch = math.ceil(samples_per_epoch / batch_size)

In [ ]:
max_f1 = 0
for epoch in range(200):
    print("Epoch {}".format(epoch))
    final_model.fit_generator(sequential_generator(filename = train_file, batch_size = batch_size, check = check_for_generator, 
                                             labels2Idx= labels2Idx,tokenize= tokenize),
                        steps_per_epoch= steps_per_epoch, epochs=1,)

    testset_features = np.zeros((len(dev_sentences), nb_sequence_length, nb_embedding_dims))
    for i in range(len(dev_sentences)):
        testset_features[i] = process_features(dev_sentences[i], nb_sequence_length, nb_embedding_dims)
    results = final_model.predict(testset_features)


    #             idx2Label = {0 : "OTHER", 1 : "OFFENSIVE"}
    predLabels = results.argmax(axis=-1)
    devLabels = dev_labels
    f1 = f1_score(devLabels, predLabels, average='macro') # offensive is the major class. So other is minor
    r = recall_score(devLabels, predLabels, average='macro')
    p = precision_score(devLabels, predLabels, average='macro')
    a = accuracy_score(devLabels, predLabels)
    
    if f1> max_f1:
        final_model.save_weights(weights_file)
        with open(log_file,'a+') as f:
            text = str(epoch)+', a: '+str(a) +', f1:' +str(f1) +'\n'
            f.write(text)
        max_f1 = f1

    print(a,f1)

In [ ]:
model = compile_model_bilstm_3cnn(n_labels)
model.load_weights('/home1/zishan/raghav/weights/tl_bilstm_3cnn_dropout=0.8_suf_bu.h5')

In [ ]:
testset_features = np.zeros((len(dev_sentences), nb_sequence_length, nb_embedding_dims))
for i in range(len(dev_sentences)):
    testset_features[i] = process_features(dev_sentences[i], nb_sequence_length, nb_embedding_dims)
results = final_model.predict(testset_features)


#             idx2Label = {0 : "OTHER", 1 : "OFFENSIVE"}
predLabels = results.argmax(axis=-1)
devLabels = dev_labels
f1 = f1_score(devLabels, predLabels, average='micro') # offensive is the major class. So other is minor
#     r = recall_score(devLabels, predLabels, average='macro')
#     p = precision_score(devLabels, predLabels, average='macro')
a = accuracy_score(devLabels, predLabels)
print(f1)